### Data Prep

In [21]:
import pandas as pd

input_file_path = r".\dive_sites_details.xlsx"

# Step 1: Read the Excel file
dive_sites_data = pd.read_excel(input_file_path)

# Step 2: Add an iterated ID column
dive_sites_data['ID'] = range(1, len(dive_sites_data) + 1)

# Step 3: Separate the Location into Latitude and Longitude
dive_sites_data[['Latitude', 'Longitude']] = dive_sites_data['Location'].str.split(',', expand=True)
dive_sites_data['Latitude'] = dive_sites_data['Latitude'].astype(float)
dive_sites_data['Longitude'] = dive_sites_data['Longitude'].astype(float)

# Step 4: Reorder the columns
dive_sites_data = dive_sites_data[['ID', 'Latitude','Description', 'Longitude', 'Title', 'Image', 'URL', 'Dive Types', 'Common Sightings', 'Maximum Depth', 'Rating']]

# Step 5: Separate the Dive Types
dive_sites_data['Dive Types'] = dive_sites_data['Dive Types'].str.split(',')
dive_sites_data['Dive Types'] = dive_sites_data['Dive Types'].apply(
    lambda x: [s.strip() for s in x if isinstance(s, str) and s.strip()] if isinstance(x, list) else []
)

dive_types_list = dive_sites_data.explode('Dive Types')[['Dive Types']].drop_duplicates().reset_index(drop=True)

connections = dive_sites_data.explode('Dive Types')[['ID', 'Dive Types']]
connections = connections[connections['Dive Types'].str.strip() != '']

# Step 6: Separate the Common Sightings
dive_sites_data['Common Sightings'] = dive_sites_data['Common Sightings'].str.split(',')
dive_sites_data['Common Sightings'] = dive_sites_data['Common Sightings'].apply(
    lambda x: [s.strip() for s in x if isinstance(s, str) and s.strip()] if isinstance(x, list) else []
)

dive_sites_data['Common Sightings'] = dive_sites_data['Common Sightings'].apply(lambda x: x if isinstance(x, list) else [])

common_sightings_list = dive_sites_data.explode('Common Sightings')[['Common Sightings']].drop_duplicates().reset_index(drop=True)

sightings_connections = dive_sites_data.explode('Common Sightings')[['ID', 'Common Sightings']]
sightings_connections = sightings_connections[sightings_connections['Common Sightings'].notna() & (sightings_connections['Common Sightings'].str.strip() != '')]

# Step 7: Drop Common Sightings and Dive Types from the original table
dive_sites_data = dive_sites_data.drop(columns=['Common Sightings', 'Dive Types'])

# Step 8: Save all DataFrames to an Excel file
output_file_path = r".\dive_sites_data.xlsx"
with pd.ExcelWriter(output_file_path) as writer:
    dive_sites_data.to_excel(writer, sheet_name='Dive Sites', index=False)
    connections.to_excel(writer, sheet_name='Dive Types Connections', index=False)
    sightings_connections.to_excel(writer, sheet_name='Common Sightings Connections', index=False)

print("Excel file with all tables has been saved successfully!")


FileNotFoundError: [Errno 2] No such file or directory: '.\\dive_sites_details.xlsx'

### Data Analysis

#### Clustering Dive Spots based on Location

In [10]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

df = pd.read_excel("dive_sites_data.xlsx")

# Lade das Shape-File mit den Regionen
shape_data = gpd.read_file("World_24NM_v4_20231025/eez_24nm_v4.shp")


In [11]:
shape_data

,MRGID,GEONAME,POL_TYPE,MRGID_TER1,TERRITORY1,MRGID_SOV1,SOVEREIGN1,ISO_TER1,X_1,Y_1,MRGID_EEZ,AREA_KM2,ISO_SOV1,UN_SOV1,UN_TER1,geometry
0,49243,Belgian 24 NM,24NM,14,Belgium,14,Belgium,BEL,2.65644,51.48898,3293,1156,BEL,56,56.0,"POLYGON ((2.30586 51.42951, 2.2866 51.46609, 2..."
1,49245,German 24 NM,24NM,2101,Germany,2101,Germany,DEU,9.77441,54.47629,5669,7900,DEU,276,276.0,"MULTIPOLYGON (((11.65639 54.37656, 11.65932 54..."
2,49246,Albanian 24 NM,24NM,2153,Albania,2153,Albania,ALB,18.99066,40.90174,5670,3645,ALB,8,8.0,"POLYGON ((19.30431 40.06083, 19.26442 40.06675..."
3,49247,Bulgarian 24 NM,24NM,2174,Bulgaria,2174,Bulgaria,BGR,28.53547,42.87690,5672,5044,BGR,100,100.0,"POLYGON ((28.94532 43.16093, 28.93992 43.15666..."
4,49248,Croatian 24 NM,24NM,2160,Croatia,2160,Croatia,HRV,15.37676,43.39258,5673,14317,HRV,191,191.0,"MULTIPOLYGON (((17.53329 42.52311, 17.54152 42..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,49439,Timorese 24 NM,24NM,8757,East Timor,8757,East Timor,TLS,126.49774,-9.14818,8758,6870,TLS,626,626.0,"MULTIPOLYGON (((126.65585 -8.21505, 126.65923 ..."
216,62592,Mauritian 24 NM (Chagos Archipelago),24NM,8616,Chagos Archipelago,8614,Republic of Mauritius,None,71.95624,-6.27622,62589,22093,MUS,480,NaN,"POLYGON ((72.82234 -4.95162, 72.82407 -4.95469..."
217,49352,Norwegian 24 NM (Bouvet),24NM,8634,Bouvet,2252,Norway,BVT,3.35301,-54.42101,8399,5313,NOR,578,74.0,"POLYGON ((3.99403 -54.18634, 3.9948 -54.18699,..."
218,49355,British 24 NM (Bermuda),24NM,8636,Bermuda,2208,United Kingdom,BMU,-64.80768,32.35403,8402,6649,GBR,826,60.0,"POLYGON ((-64.75086 32.87028, -64.7508 32.8702..."


In [12]:
# Entferne " 24 NM" am Ende jedes Eintrags in der Spalte "GEONAME"
shape_data['GEONAME'] = shape_data['GEONAME'].str.replace(" 24 NM$", "", regex=True)

# Überprüfe das Ergebnis
print(shape_data[['GEONAME']].head())

# Konvertiere die Shape-File-Koordinaten und den DataFrame in das gleiche CRS (Coordinate Reference System)
shape_data = shape_data.to_crs("EPSG:4326")  # WGS 84 Koordinatensystem für lat/lon
gdf_spots = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Longitude'], df['Latitude']), crs="EPSG:4326")

     GEONAME
0    Belgian
1     German
2   Albanian
3  Bulgarian
4   Croatian


In [17]:
# Funktion zur Zuordnung der Region zu jedem Tauchspot
def get_region(point):
    for _, region in shape_data.iterrows():
        if region.geometry.contains(point):
            print(point)
            return region['GEONAME']
    return "undefined"

# Neue Spalte mit Regionenwerten hinzufügen
gdf_spots['Region'] = gdf_spots['geometry'].apply(get_region)

# Ergebnis anzeigen
gdf_spots[['ID', 'Title', 'Region']]

POINT (35.67535 -23.45065)
POINT (35.66093 -23.51237)
POINT (35.65475 -23.53755)
POINT (35.64926 -23.55643)
POINT (98.83132 7.15942)
POINT (12.31773 41.42162)
POINT (98.82077 7.15217)
POINT (-110.12558 24.657)
POINT (-49.42062 -29.43601)
POINT (98.82 7.1518)
POINT (149.07383 -19.80749)
POINT (-28.99083 38.33528)
POINT (-87.00662 30.04258)
POINT (101.399 12.25776)
POINT (-81.98667 25.91)
POINT (-81.98667 25.90917)
POINT (35.8374 -23.53881)
POINT (-34.55685 -8.19199)
POINT (-34.59457 -8.22372)
POINT (-73.50736 40.25662)


,ID,Title,Region
0,1,HMS Maori,undefined
1,2,USAT Liberty Shipwreck,undefined
2,3,Ped,undefined
3,4,Manta Point,undefined
4,5,THE HOLE / GREEN BAY CAVES,undefined
...,...,...,...
4390,4391,"Lekuan 1, 2, 3",undefined
4391,4392,Moc-Che,undefined
4392,4393,Fish Market,undefined
4393,4394,The Zenobia Wreck,undefined


In [20]:
gdf_spots[gdf_spots["Region"] != "undefined"]

,ID,Latitude,Longitude,Title,Image,URL,Maximum Depth,Rating,geometry,Region
426,427,-23.45065,35.67535,Colosseum,https://d2p1cf6997m1ir.cloudfront.net/media/th...,https://www.padi.com/dive-site/mozambique/colo...,NaN,NaN,POINT (35.67535 -23.45065),Mozambican
427,428,-23.51237,35.66093,Banners,https://d2p1cf6997m1ir.cloudfront.net/media/th...,https://www.padi.com/dive-site/mozambique/bann...,NaN,NaN,POINT (35.66093 -23.51237),Mozambican
429,430,-23.53755,35.65475,The Office,https://d2p1cf6997m1ir.cloudfront.net/media/th...,https://www.padi.com/dive-site/mozambique/the-...,NaN,NaN,POINT (35.65475 -23.53755),Mozambican
430,431,-23.55643,35.64926,Reggies,https://d2p1cf6997m1ir.cloudfront.net/media/th...,https://www.padi.com/dive-site/mozambique/regg...,NaN,NaN,POINT (35.64926 -23.55643),Mozambican
514,515,7.15942,98.83132,Hin Muang,https://d2p1cf6997m1ir.cloudfront.net/media/th...,https://www.padi.com/dive-site/thailand/hin-mu...,NaN,NaN,POINT (98.83132 7.15942),Thai
1242,1243,41.42162,12.31773,Area Marina Protetta di Tor Paterno - Roma,https://d2p1cf6997m1ir.cloudfront.net/media/th...,https://www.padi.com/dive-site/italy/area-mari...,NaN,NaN,POINT (12.31773 41.42162),Italian
1474,1475,7.15217,98.82077,Hin Daeng - Hin Muang,https://d2p1cf6997m1ir.cloudfront.net/media/th...,https://www.padi.com/dive-site/thailand/hin-da...,NaN,NaN,POINT (98.82077 7.15217),Thai
1590,1591,24.65700,-110.12558,El Bajo (Marisla),https://d2p1cf6997m1ir.cloudfront.net/media/th...,https://www.padi.com/dive-site/mexico/el-bajo-...,NaN,NaN,POINT (-110.12558 24.657),Mexican
2140,2141,-29.43601,-49.42062,PARCEL DE TORRES,https://d2p1cf6997m1ir.cloudfront.net/media/th...,https://www.padi.com/dive-site/brazil/parcel-d...,108 feet / 33 meters,NaN,POINT (-49.42062 -29.43601),Brazilian
2191,2192,7.15180,98.82000,Hin Daeng,https://d2p1cf6997m1ir.cloudfront.net/media/th...,https://www.padi.com/dive-site/thailand/hin-da...,98 feet / 30 meters,NaN,POINT (98.82 7.1518),Thai


#### Clustering Dive Spots based on Specifications

#### Clustering Users based on their existing Dives and Reviews

#### Combining Dive Spot Clustering and User Data

#### Build a Recommender Function